In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI管道：使用KFP SDK的管道控制结构

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/control_flow_kfp.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fpipelines%2Fcontrol_flow_kfp.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab企业版中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/pipelines/control_flow_kfp.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在工作台中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/control_flow_kfp.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub中查看
    </a>
  </td>
</table>

## 概述

本笔记本演示了如何使用 [Kubeflow Pipelines (KFP) SDK](https://www.kubeflow.org/docs/components/pipelines/v2/) 构建使用控制结构的 Vertex AI 管道。

了解更多关于 [Vertex AI 管道](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)。

### 目标

在本教程中，您将学习如何使用 KFP SDK，该工具使用循环和条件语句，包括嵌套示例，来构建管道。

本教程使用以下 Google Cloud ML 服务：

- Vertex AI Pipelines

执行的步骤包括：

- 使用控制流组件创建 KFP 管道
- 编译 KFP 管道
- 使用 Vertex AI Pipelines 执行 KFP 管道

费用

本教程使用Google Cloud的以下可计费组件:

* Vertex AI
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成费用估算。

开始吧

### 为 Python 安装 Vertex AI SDK 和其他所需的软件包

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform  \
                                 google-cloud-storage \
                                 kfp \
                                 google-cloud-pipeline-components

重新启动运行时（仅在Colab中）

要使用新安装的包，必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核即将重新启动。在继续下一步之前，请等待完成。⚠️</b>
</div>

### 认证您的笔记本环境（仅限Colab）

在Google Colab上认证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息

要开始使用Vertex AI，您必须拥有现有的Google Cloud项目。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶

创建一个存储桶，用于存储诸如数据集之类的中间产品。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在： 运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

#### 服务账户

**如果您不知道您的服务账户**，请使用`gcloud`命令通过执行下面的第二个单元格来创建您的服务账户。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### 为Vertex AI Pipelines设置服务账户访问权限

运行以下命令，以授予您的服务账户在上一步创建的存储桶中读取和写入管道工件的权限。您只需要每个服务账户运行一次这些命令。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 导入库并定义常量

In [ ]:
import json

from google.cloud import aiplatform
from kfp import compiler, dsl
from kfp.dsl import component

#### Vertex AI Pipelines 常量

为 Vertex AI Pipelines 设置以下常量：

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/control".format(BUCKET_URI)

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## 定义管道组件

以下示例定义了三个简单的管道组件：

- 一个生成字典列表并返回一个字符串化的 JSON 的组件。
（**注意**：此组件在组件函数定义中需要 `import json`）
- 一个仅打印其输入字符串的组件
- 一个执行 '抛硬币' 操作并返回 `heads` 或 `tails` 的组件。

In [ ]:
@component
def args_generator_op() -> str:
    import json

    return json.dumps(
        [{"cats": "1", "dogs": "2"}, {"cats": "10", "dogs": "20"}],
        sort_keys=True,
    )


@component
def print_op(msg: str):
    print(msg)


@component
def flip_coin_op() -> str:
    """Flip a coin and return heads or tails randomly."""
    import random

    result = "heads" if random.randint(0, 1) == 0 else "tails"
    return result

定义一个使用控制结构的流水线

以下示例定义了一个流水线组件，展示了如何使用`dsl.Condition`和`dsl.ParallelFor`。

`json_string`的默认值是一个转换为字符串的嵌套JSON列表。根据流水线定义，循环和条件表达式将此字符串作为列表处理，并访问列表项和子项。
`args_generator_op`输出的列表也是如此。

In [ ]:
@dsl.pipeline(
    name="control",
    pipeline_root=PIPELINE_ROOT,
)
def pipeline(
    json_string: str = json.dumps(
        [
            {
                "snakes": "anaconda",
                "lizards": "anole",
                "bunnies": [{"cottontail": "bugs"}, {"cottontail": "thumper"}],
            },
            {
                "snakes": "cobra",
                "lizards": "gecko",
                "bunnies": [{"cottontail": "roger"}],
            },
            {
                "snakes": "boa",
                "lizards": "iguana",
                "bunnies": [
                    {"cottontail": "fluffy"},
                    {"fuzzy_lop": "petunia", "cottontail": "peter"},
                ],
            },
        ],
        sort_keys=True,
    )
):

    flip1 = flip_coin_op()

    with dsl.Condition(
        flip1.output != "no-such-result", name="alwaystrue"
    ):  # always true

        args_generator = args_generator_op()
        with dsl.ParallelFor(args_generator.output) as item:
            print_op(msg=json_string)

            with dsl.Condition(flip1.output == "heads", name="heads"):
                print_op(msg=item.cats)

            with dsl.Condition(flip1.output == "tails", name="tails"):
                print_op(msg=item.dogs)

    with dsl.ParallelFor(json_string) as item:
        with dsl.Condition(item.snakes == "boa", name="snakes"):
            print_op(msg=item.snakes)
            print_op(msg=item.lizards)
            print_op(msg=item.bunnies)

    # it is possible to access sub-items
    with dsl.ParallelFor(json_string) as item:
        with dsl.ParallelFor(item.bunnies) as item_bunnies:
            print_op(msg=item_bunnies.cottontail)

## 编译管道

接下来，编译管道。

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="control_pipeline.yaml"
)

运行管道

接下来，运行管道。

In [ ]:
DISPLAY_NAME = "control"

job = aiplatform.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="control_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
)

job.run()

点击生成的链接以查看在云控制台中运行的情况。

在Google云控制台中，您可以单击管道DAG节点以展开或折叠它们。下面是DAG的部分展开视图（单击图像查看更大的版本）。

<a href="https://storage.googleapis.com/amy-jo/images/mp/control_flow_dag.png" target="_blank"><img src="https://storage.googleapis.com/amy-jo/images/mp/control_flow_dag.png" width="95%"/></a>

您可以看到，例如，“heads”条件通过，而“tails”条件预期失败。

清理工作

要清理本项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket = False

# Delete the pipeline job
job.delete()

# Delete the locally generated files
! rm control_pipeline.yaml

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI